In [1]:
import psycopg2
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd

In [2]:
state_abbreviations = pd.read_csv('state_abbreviations.csv')

In [3]:
covid_cases = pd.read_csv("covid-cases-by-state.csv")
covid_cases = covid_cases[['State', 'Total Cases', 'Total Deaths', 'Cases per million', 'Deaths per million', 'Tests', 'Tests per million', 'Total Population']]
covid_cases['State'].replace(state_abbreviations['State'].tolist(), state_abbreviations['Code'].tolist(), inplace=True)
covid_cases.head()

,State,Total Cases,Total Deaths,Cases per million,Deaths per million,Tests,Tests per million,Total Population
0,CA,"2,345,657","26,294","59,365",665,"33,058,311","836,660","39,512,223"
1,TX,"1,784,569","28,506","61,546",983,"15,832,807","546,036","28,995,881"
2,FL,"1,323,315","21,673","61,613","1,009","15,703,599","731,157","21,477,737"
3,NY,"1,031,053","38,145","53,001","1,961","25,504,313","1,311,036","19,453,561"
4,IL,"970,590","18,173","76,594","1,434","13,374,665","1,055,465","12,671,821"


In [4]:
prf = pd.read_csv("HHS_Provider_Relief_Fund.csv")
prf['Payment']=(prf['Payment'].replace( '[\$,)]','', regex=True )
                   .replace( '[(]','-',   regex=True ).astype(float))
prf.head()


,Provider Name,State,City,Payment
0,BRANDON ASTIN DMD LLC,AK,ANCHOR POINT,19003.0
1,ELIZABETH WATNEY,AK,ANCHOR POINT,724.0
2,A HAND UP BEHAVIOR SERVICES,AK,ANCHORAGE,1191.0
3,A JOINT EFFORT PHYSICAL THERAPY,AK,ANCHORAGE,11641.0
4,"AA PAIN CLINIC, INC.",AK,ANCHORAGE,69976.0


In [5]:
payments_by_state = prf[['State', 'Payment']].groupby('State').sum()
payments_by_state.reset_index(inplace=True)



In [6]:
from config import local_db_user, local_db_pwd, local_db_endpoint, local_db_port, local_db_name

In [7]:
# local Postgres DB Connection
local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")

# Create a local database engine connection
local_conn = local_engine.connect()

# Create local database
try:
    local_conn.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {local_db_name}")
except Exception as e:
    print(e)
    
# Connect to local database
try:
    local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}/{local_db_name}")
    local_conn = local_engine.connect()
except Exception as e:
    print(e)

#confirm tables
local_engine.table_names()

#upload dataframe to database
prf.to_sql(name='prf', if_exists='replace', con=local_conn, index=False)



In [8]:
payments_by_state.to_sql(name='payments_by_state', if_exists='replace', con=local_conn, index=False)

In [11]:
covid_cases.to_sql(name='covid_cases', if_exists='replace', con=local_conn, index=False)

In [9]:
sql = """
ALTER TABLE payments_by_state ADD PRIMARY KEY ("State");
"""
with local_engine.begin() as conn:
    conn.execute(sql)

In [10]:
sql = """
ALTER TABLE prf 
ADD CONSTRAINT state_constraint 
FOREIGN KEY ("State") 
REFERENCES payments_by_state ("State");;
"""
with local_engine.begin() as conn:
    conn.execute(sql)